In [1]:
import os
from dotenv import load_dotenv
from functools import lru_cache
from crewai import LLM
# Load environment variables from .env file
load_dotenv()

# Set the environment variables
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

print("API Keys loaded successfully.")

API Keys loaded successfully.


In [2]:
@lru_cache(maxsize=1)
def initialize_llm():
    """Initialize and cache the LLM instance to avoid repeated initializations."""
    return LLM(
        model="gemini/gemini-2.0-flash",
        provider="google",
        api_key=GEMINI_API_KEY
    )

In [3]:
import requests
import json
from crewai.tools import tool          # decorator
from crewai_tools import SerperDevTool # web-search tool

# Initialize the web search tool
search_tool = SerperDevTool()

# Tool 1: Human Input Tool
# This tool pauses the execution and asks for human input.
@tool("Human Input Tool")
def human_input_tool(question: str) -> str:
    """Asks a human for input. The agent should use this to ask for a budget if one is not provided."""
    return input(f"\n{question}\n")

def geocode_city(city: str) -> tuple[float, float] | None:
    url = "https://geocoding-api.open-meteo.com/v1/search"
    resp = requests.get(url, params={"name": city, "count": 1, "language": "en"})
    resp.raise_for_status()
    results = resp.json().get("results")
    if results:
        return results[0]["latitude"], results[0]["longitude"]
    return None

# Tool 2: Weather Tool (Updated for Forecast)
bad_weather_codes = [51, 53, 55, 56, 57, 61, 63, 65, 66, 67, 71, 73, 75, 77, 80, 81, 82, 85, 86, 95, 96, 99]
desc_map = {
    0: "clear sky", 1: "mainly clear", 2: "partly cloudy", 3: "overcast", 45: "foggy", 51: "light drizzle", 
    61: "rain", 71: "snow", 95: "thunderstorm"
}

@tool("Weather Tool")
def open_meteo_weather_tool(city: str, start_date: str, end_date: str) -> str:
    """Returns weather forecast for a city between start_date and end_date using Open-Meteo."""
    coords = geocode_city(city)
    if not coords:
        return f"Sorry, I couldn’t find coordinates for {city}."
    lat, lon = coords
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "daily": "temperature_2m_max,temperature_2m_min,weathercode",
        "start_date": start_date,
        "end_date": end_date,
        "timezone": "auto"
    }
    try:
        r = requests.get(url, params=params, timeout=8)
        r.raise_for_status()
        data = r.json()
        daily = data["daily"]
        forecast_lines = [f"Weather forecast for {city.title()} from {start_date} to {end_date}:"]
        bad_weather_dates = []
        for i in range(len(daily["time"])):
            date = daily["time"][i]
            max_temp = daily["temperature_2m_max"][i]
            min_temp = daily["temperature_2m_min"][i]
            code = daily["weathercode"][i]
            desc = desc_map.get(code, "unknown")
            forecast_lines.append(f"- {date}: {min_temp}°C to {max_temp}°C, {desc}")
            if code in bad_weather_codes:
                bad_weather_dates.append(date)
        if bad_weather_dates:
            forecast_lines.append("\nNote: Bad weather (rain, snow, or thunderstorms) expected on: " + ", ".join(bad_weather_dates))
        return "\n".join(forecast_lines)
    except Exception as e:
        return f"Error fetching Open-Meteo data: {e}"

# Tool 3: Currency Conversion Tool
@tool("Currency Conversion Tool")
def currency_conversion_tool(from_currency: str, to_currency: str) -> str:
    """Converts a value from one currency to another using a free API."""
    try:
        # Using exchangerate-api.com's free tier
        url = f"https://open.er-api.com/v6/latest/{from_currency}"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        rate = data['rates'][to_currency]
        return f"The conversion rate from {from_currency} to {to_currency} is {rate:.2f}."
    except Exception as e:
        return f"Error converting currency: {e}. Ensure currency codes are correct (e.g., USD, EUR, LKR)."


print("Tools created successfully.")

Tools created successfully.


/Users/nithilathawalampitiya/Documents/Projects/TravelAgent/myenv-crew/lib/python3.12/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [4]:
# Define country to currency mapping
country_to_currency = {
    'Sri Lanka': 'LKR',
    'United States': 'USD',
    'United Kingdom': 'GBP',
    # Add more as needed
}

# --- Define User Inputs ---
location = 'Mirissa, Sri Lanka'
interests = 'entertainment, beach and villa'
budget = '200 USD'
num_people = 4 
travel_dates = '2025-08-05 to 2025-08-06'  # Static travel dates
preferred_currency = ''  # Leave empty for local currency, or set to 'GBP', etc.

# Determine local currency
country = location.split(',')[-1].strip()
local_currency = country_to_currency.get(country, 'USD')
target_currency = preferred_currency if preferred_currency else local_currency

# Initialize LLM
llm_model = initialize_llm()

In [5]:
from crewai import Agent

# Agent 1: Budget Inquiry Agent
budget_agent = Agent(
    role='Budget Inquiry Specialist',
    goal='Determine the user’s trip budget, using the provided value or asking if not given.',
    backstory='A friendly expert in gathering client financial details.',
    tools=[human_input_tool],
    llm=llm_model,
    verbose=True
)

# Agent 2: Local Data Agent
local_data_agent = Agent(
    role="Local Data Specialist",
    goal="Fetch weather and currency data for the travel destination.",
    backstory="An analyst providing real-time travel insights.",
    tools=[open_meteo_weather_tool, currency_conversion_tool],
    llm=llm_model,
    verbose=True
)

# Agent 3: Web Search Agent (City Expert)
city_expert_agent = Agent(
    role='Expert City Researcher',
    goal='Find activities, landmarks, and restaurants based on interests, budget, and weather.',
    backstory='A travel enthusiast who finds the best spots tailored to your needs.',
    tools=[search_tool],
    llm=llm_model,
    verbose=True
)

# Agent 4: Budget Verifier Agent
budget_verifier_agent = Agent(
    role='Budget Verification Analyst',
    goal='Critically analyze the researched activities and their estimated costs against the user-provided budget. Provide a clear "go" or "no-go" verdict with justification.',
    backstory='A meticulous financial analyst with a knack for sniffing out hidden costs and ensuring travel plans are financially sound. You are firm but fair.',
    tools=[],
    llm=llm_model,
    allow_delegation=False,
    verbose=True
)

# Agent 5: Travel Concierge Agent
travel_concierge_agent = Agent(
    role='Head Travel Concierge',
    goal='Synthesize all gathered information into a cohesive, beautifully formatted travel itinerary with weather insights and converted costs.',
    backstory='A world-class concierge from a five-star hotel, known for creating personalized and delightful travel experiences.',
    tools=[currency_conversion_tool],  # Added for cost conversion
    llm=llm_model,
    allow_delegation=False,
    verbose=True
)

print("Agents defined successfully.")

Agents defined successfully.


In [6]:
from crewai import Task

# Task 1: Ask for budget if not provided
task_get_budget = Task(
    description=f"Determine the user's budget for the trip to {location}. If provided in inputs as '{budget}', use it. Otherwise, ask the user for their total budget in USD using the Human Input Tool. Interests: {interests}.\
        Only use the Human Input Tool if {budget} is empty or null.",
    expected_output="The user's budget in USD (e.g., '1500 USD').",
    agent=budget_agent,
    skip_if=lambda: budget != '' # Add this line to skip the task if a budget is provided
)

# Task 2: Get local data (weather forecast and currency conversion)
task_get_local_data = Task(
    description=f"""Fetch the weather forecast and the USD to local currency conversion rate for the trip to {location}.

    The travel dates are: {travel_dates}.

    You MUST use the Weather Tool with the exact start and end dates from the provided travel dates.
    Do not use 'today' or any other made-up date. First, call the currency conversion tool, then call the weather tool.
    """,
    expected_output="A summary of the weather forecast for the specified dates and the USD to local currency conversion rate.",
    agent=local_data_agent
)

# Task 3: Find city information
task_find_city_info = Task(
    description=f"Using the web search tool, find 3-4 top attractions, activities, and restaurants in {location} for a group of {num_people} people that align with interests: {interests}. Use the budget and weather from context to prioritize options within budget and suitable for the weather (e.g., outdoor for clear skies, indoor for rain). Include descriptions and costs in USD if possible.\
        Strictly give outputs that are less than or equal to the budget and also it should be close to the budget too. When estimating costs (for activities, food, etc.), provide a total estimated cost for the entire group of {num_people}.",
    expected_output="A list of attractions, activities, and restaurants with descriptions and costs, stating the city.",
    agent=city_expert_agent,
    context=[task_get_budget, task_get_local_data]
)

# Task 4: Verify the budget
task_verify_budget = Task(
    description="Analyze the city research and budget from context for a group of {num_people}. Sum the total estimated costs for the group and compare them to the total budget ('{budget}'). Provide a clear verdict on whether the plan is financially feasible for the group.",
    expected_output="A budget feasibility verdict with total estimated costs vs. budget.",
    agent=budget_verifier_agent,
    context=[task_get_budget, task_find_city_info]
)

# Task 5: Compile the final report
task_compile_report = Task(
    description=f"Create a final, human-readable travel itinerary for {num_people} people for a trip to {location} from {travel_dates}. If {travel_dates} are inclusive of 2 or more dates, You must suggest a place to stay the night with the given budget. Combine the list of activities and budget verification verdict into a well-formatted report. Convert all costs from USD to {target_currency} using the Currency Conversion Tool (do not show the conversion rate, only the converted costs). Use the weather forecast from the local data task to provide insights: if bad weather is expected on certain days, suggest indoor activities or adjusting travel dates by +/- 5 days. Make it exciting and professional.",
    expected_output=f"A complete, beautiful markdown report with the travel plan and budget analysis, with costs in {target_currency}.",
    agent=travel_concierge_agent,
    context=[task_verify_budget, task_get_local_data, task_find_city_info]
)

print("Tasks created successfully.")

Tasks created successfully.


In [7]:
from crewai import Crew, Process
from IPython.display import Markdown

# Create the Crew
travel_crew = Crew(
    agents=[budget_agent, local_data_agent, city_expert_agent, budget_verifier_agent, travel_concierge_agent],
    tasks=[task_get_budget, task_get_local_data, task_find_city_info, task_verify_budget, task_compile_report],
    process=Process.sequential,
    verbose=True
)

# Kick off the crew's work!
result = travel_crew.kickoff(inputs={'location': location, 'interests': interests, 'budget': budget, 'travel_dates': travel_dates, 'num_people': num_people})

# Print the final result
if hasattr(result, 'raw') and isinstance(result.raw, str):
    display(Markdown(result.raw))
else:
    print(f"Error: Expected a CrewOutput object with a 'raw' attribute containing a string, but got {type(result)}")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 109fd144-62bf-4fc3-9d86-399a4426020a                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Budget Inquiry Specialist                                                                               │
│                                                                                                                 │
│  Task: Determine the user's budget for the trip to Mirissa, Sri Lanka. If provided in inputs as '200 USD', use  │
│  it. Otherwise, ask the user for their total budget in USD using the Human Input Tool. Interests:               │
│  entertainment, beach and villa.        Only use the Human Input Tool if 200 USD is empty or null.              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Budget Inquiry Specialist                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  200 USD                                                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ceecd3d5-6167-4909-84ec-9538a9fbd9ef                                                                     │
│  Agent: Budget Inquiry Specialist                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Data Specialist                                                                                   │
│                                                                                                                 │
│  Task: Fetch the weather forecast and the USD to local currency conversion rate for the trip to Mirissa, Sri    │
│  Lanka.                                                                                                         │
│                                                                                                                 │
│      The travel dates are: 2025-08-05 to 2025-08-06.                                                            │
│                                                                                                                 │
│      You MUST use the Weather Tool with the exact start and end dates from the provided travel dates.           │
│      Do not use 'today' or any other made-up date. First, call the currency conversion tool, then call the      │
│  weather tool.                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  The conversion rate from USD to LKR is 301.90.                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Head Travel Concierge                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # Mirissa Getaway: A Tailored Itinerary for Your Group of 4 (August 5-6, 2025)                                 │
│                                                                                                                 │
│  Welcome to Mirissa! Get ready for a delightful escape filled with stunning beaches, delicious food, and        │
│  unforgettable experiences. This itinerary is designed with your preferences (entertainment, beach, villa       │
│  experience) and budget in mind, while also considering the weather forecast.                                   │
│                                                                                                                 │
│  **Weather Insights:**                                                                                          │
│                                                                                                                 │
│  *   **August 5, 2025:** Overcast skies, with temperatures ranging from 26.4°C to 29.4°C.                       │
│  *   **August 6, 2025:** Light drizzle expected, with temperatures between 23.9°C and 29.5°C.                   │
│                                                                                                                 │
│  *Recommendation:* Given the light drizzle forecast for August 6th, we've included some flexible activity       │
│  options.                                                                                                       │
│                                                                                                                 │
│  **Accommodation Recommendation:**                                                                              │
│                                                                                                                 │
│  As your trip is inclusive of 2 dates, we suggest staying at a beachfront villa for the night to fully immerse  │
│  yourself in the Mirissa experience. There are several options to choose from that fit various budgets. Please  │
│  let us know if you need help with booking.                                                                     │
│                                                                                                                 │
│  ## Day 1: August 5, 2025 - Coastal Views and Evening Vibes                                                     │
│                                                                                                                 │
│  *   **Morning (Free): Arrival & Beach Exploration**                                                            │
│                                                                                                                 │
│      *   Settle into your accommodation and head to Mirissa Beach. Take a leisurely stroll along the shore,     │
│  soak up the atmosphere, and enjoy the ocean breeze.                                                            │
│  *   **Afternoon (Free): Coconut Tree Hill**                                                                    │
│                                                                                                                 │
│      *   Visit Coconut Tree Hill, a picturesque spot pe

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 5aea0a9e-69e6-453e-a40a-d58730481e16                                                                     │
│  Agent: Head Travel Concierge                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 109fd144-62bf-4fc3-9d86-399a4426020a                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # Mirissa Getaway: A Tailored Itinerary for Your Group of 4 (August 5-6, 2025)                                 │
│                                                                                                                 │
│  Welcome to Mirissa! Get ready for a delightful escape filled with stunning beaches, delicious food, and        │
│  unforgettable experiences. This itinerary is designed with your preferences (entertainment, beach, villa       │
│  experience) and budget in mind, while also considering the weather forecast.                                   │
│                                                                                                                 │
│  **Weather Insights:**                                                                                          │
│                                                                                                                 │
│  *   **August 5, 2025:** Overcast skies, with temperatures ranging from 26.4°C to 29.4°C.                       │
│  *   **August 6, 2025:** Light drizzle expected, with temperatures between 23.9°C and 29.5°C.                   │
│                                                                                                                 │
│  *Recommendation:* Given the light drizzle forecast for August 6th, we've included some flexible activity       │
│  options.                                                                                                       │
│                                                                                                                 │
│  **Accommodation Recommendation:**                                                                              │
│                                                                                                                 │
│  As your trip is inclusive of 2 dates, we suggest staying at a beachfront villa for the night to fully immerse  │
│  yourself in the Mirissa experience. There are several options to choose from that fit various budgets. Please  │
│  let us know if you need help with booking.                                                                     │
│                                                                                                                 │
│  ## Day 1: August 5, 2025 - Coastal Views and Evening Vibes                                                     │
│                                                                                                                 │
│  *   **Morning (Free): Arrival & Beach Exploration**                                                            │
│                                                                                                                 │
│      *   Settle into your accommodation and head to Mirissa Beach. Take a leisurely stroll along the shore,     │
│  soak up the atmosphere, and enjoy the ocean breeze.                                                            │
│  *   **Afternoon (Free): Coconut Tree Hill**                                                                    │
│                                                       

```markdown
# Mirissa Getaway: A Tailored Itinerary for Your Group of 4 (August 5-6, 2025)

Welcome to Mirissa! Get ready for a delightful escape filled with stunning beaches, delicious food, and unforgettable experiences. This itinerary is designed with your preferences (entertainment, beach, villa experience) and budget in mind, while also considering the weather forecast.

**Weather Insights:**

*   **August 5, 2025:** Overcast skies, with temperatures ranging from 26.4°C to 29.4°C.
*   **August 6, 2025:** Light drizzle expected, with temperatures between 23.9°C and 29.5°C.

*Recommendation:* Given the light drizzle forecast for August 6th, we've included some flexible activity options.

**Accommodation Recommendation:**

As your trip is inclusive of 2 dates, we suggest staying at a beachfront villa for the night to fully immerse yourself in the Mirissa experience. There are several options to choose from that fit various budgets. Please let us know if you need help with booking.

## Day 1: August 5, 2025 - Coastal Views and Evening Vibes

*   **Morning (Free): Arrival & Beach Exploration**

    *   Settle into your accommodation and head to Mirissa Beach. Take a leisurely stroll along the shore, soak up the atmosphere, and enjoy the ocean breeze.
*   **Afternoon (Free): Coconut Tree Hill**

    *   Visit Coconut Tree Hill, a picturesque spot perfect for capturing stunning photos and enjoying panoramic views of the coastline. *Cost: Free (LKR 0)*
*   **Evening:**

    *   **Dinner at Dewmini Roti Shop:** Indulge in delicious and affordable local roti. A culinary experience you won't forget! *Cost: Approximately LKR 4,528.50 (USD 15)*

## Day 2: August 6, 2025 - Beach Relaxation and Sri Lankan Flavors

*   **Morning:**

    *   **Relaxation at Mirissa Beach:** Even with the light drizzle, enjoy the atmosphere at Mirissa Beach! Many of the beach bars and restaurants are covered.

    *   *Cost: Drinks & snacks: LKR 6,038 (USD 20)*
*   **Afternoon:**

    *   **Lunch at Oh La La:** Enjoy delicious seafood with a beautiful ocean view and refreshing mango juice. *Cost: Approximately LKR 12,076 (USD 40)*
*   **Evening:**

    *   **Dinner at Ceylon Curry House:** Experience authentic Sri Lankan curries packed with flavor. *Cost: Approximately LKR 7,547.50 (USD 25)*

## Entertainment

*   **Evening Entertainment at Doctor's House:** (Both Evenings) A popular spot for evening drinks, food, and live music. Ideal for an evening even with the weather.
    *Cost: Food and drinks for 4: LKR 15,095 (USD 50 total (Estimated, adjust based on consumption))*

**Budget Analysis:**

*   **Activities:** LKR 21,133 (USD 70)
*   **Restaurants:** LKR 24,661 (USD 80)
*   **Total Estimated Cost:** LKR 45,794 (USD 150)

**Verdict: GO**

This itinerary is financially feasible within your budget of $200 USD (LKR 60,380), leaving approximately LKR 15,095 (USD 50) for miscellaneous expenses or transport.

**Important Notes:**

*   All costs are estimates and may vary.
*   Remember to check the most current prices and availability.
*   Enjoy your amazing trip to Mirissa!
```

In [8]:
# import os
# from dotenv import load_dotenv
# from functools import lru_cache
# from crewai import LLM
# import requests
# import json
# from datetime import datetime, timedelta
# from crewai.tools import tool
# from crewai_tools import SerperDevTool
# from crewai import Agent, Task, Crew, Process

# # Load environment variables from .env file
# load_dotenv()

# # Set the environment variables
# GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
# os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
# print("API Keys loaded successfully.")

# @lru_cache(maxsize=1)
# def initialize_llm():
#     """Initialize and cache the LLM instance to avoid repeated initializations."""
#     return LLM(
#         model="gemini/gemini-2.0-flash",
#         provider="google",
#         api_key=GEMINI_API_KEY
#     )

# # Initialize the web search tool
# search_tool = SerperDevTool()

# # Tool 1: Human Input Tool
# @tool("Human Input Tool")
# def human_input_tool(question: str) -> str:
#     """Asks a human for input. The agent should use this to ask for a budget if one is not provided."""
#     return input(f"\n{question}\n")

# def geocode_city(city: str) -> tuple[float, float] | None:
#     url = "https://geocoding-api.open-meteo.com/v1/search"
#     resp = requests.get(url, params={"name": city, "count": 1, "language": "en"})
#     resp.raise_for_status()
#     results = resp.json().get("results")
#     if results:
#         return results[0]["latitude"], results[0]["longitude"]
#     return None

# def get_country_from_city(city: str) -> str | None:
#     """Get country name from city name"""
#     coords = geocode_city(city)
#     if coords:
#         lat, lon = coords
#         url = "https://geocoding-api.open-meteo.com/v1/reverse"
#         params = {"latitude": lat, "longitude": lon, "format": "json"}
#         resp = requests.get(url, params=params)
#         resp.raise_for_status()
#         results = resp.json().get("results")
#         if results:
#             return results[0].get("country")
#     return None

# def get_currency_from_country(country: str) -> str:
#     """Get currency code from country name"""
#     country_currency_map = {
#         "Sri Lanka": "LKR",
#         "United States": "USD",
#         "United Kingdom": "GBP",
#         "European Union": "EUR",
#         "Japan": "JPY",
#         "Australia": "AUD",
#         "Canada": "CAD",
#         "Switzerland": "CHF",
#         "China": "CNY",
#         "India": "INR",
#         # Add more as needed
#     }
#     return country_currency_map.get(country, "USD")

# # Tool 2: Enhanced Weather Tool
# @tool("Weather Tool")
# def enhanced_weather_tool(city: str, travel_date: str = None) -> str:
#     """
#     Returns weather forecast and recommendations for a city using Open-Meteo.
#     If travel_date is provided, focuses on that date and suggests alternatives if weather is bad.
#     """
#     coords = geocode_city(city)
#     if not coords:
#         return f"Sorry, I couldn't find coordinates for {city}."
    
#     lat, lon = coords
    
#     # Get 10-day forecast
#     url = "https://api.open-meteo.com/v1/forecast"
#     params = {
#         "latitude": lat,
#         "longitude": lon,
#         "daily": "weathercode,temperature_2m_max,temperature_2m_min,precipitation_sum,precipitation_hours",
#         "timezone": "auto",
#         "forecast_days": 10
#     }
    
#     try:
#         r = requests.get(url, params=params, timeout=8)
#         r.raise_for_status()
#         data = r.json()
        
#         daily_data = data["daily"]
#         dates = daily_data["time"]
#         weather_codes = daily_data["weathercode"]
#         max_temps = daily_data["temperature_2m_max"]
#         min_temps = daily_data["temperature_2m_min"]
#         precipitation = daily_data["precipitation_sum"]
#         precip_hours = daily_data["precipitation_hours"]
        
#         # Weather code descriptions
#         weather_desc_map = {
#             0: "Clear sky", 1: "Mainly clear", 2: "Partly cloudy",
#             3: "Overcast", 45: "Foggy", 48: "Foggy",
#             51: "Light drizzle", 53: "Drizzle", 55: "Dense drizzle",
#             56: "Freezing drizzle", 57: "Freezing drizzle",
#             61: "Light rain", 63: "Rain", 65: "Heavy rain",
#             66: "Freezing rain", 67: "Freezing rain",
#             71: "Light snow", 73: "Snow", 75: "Heavy snow",
#             77: "Snow grains", 80: "Light showers", 81: "Showers", 82: "Heavy showers",
#             85: "Light snow showers", 86: "Heavy snow showers",
#             95: "Thunderstorm", 96: "Thunderstorm", 99: "Severe thunderstorm"
#         }
        
#         # Analyze weather and provide recommendations
#         recommendations = []
        
#         if travel_date:
#             # Find the index of the travel date
#             target_date = datetime.strptime(travel_date, "%Y-%m-%d").date()
#             date_indices = [i for i, date_str in enumerate(dates) 
#                           if datetime.strptime(date_str, "%Y-%m-%d").date() == target_date]
            
#             if date_indices:
#                 idx = date_indices[0]
#                 weather_code = weather_codes[idx]
#                 weather_desc = weather_desc_map.get(weather_code, f"Weather code {weather_code}")
#                 precip = precipitation[idx]
#                 precip_hrs = precip_hours[idx]
                
#                 # Check if weather is bad (rain, snow, thunderstorm)
#                 bad_weather_codes = [51, 53, 55, 56, 57, 61, 63, 65, 66, 67, 71, 73, 75, 80, 81, 82, 85, 86, 95, 96, 99]
                
#                 if weather_code in bad_weather_codes or precip > 5 or precip_hrs > 3:
#                     recommendations.append(f"⚠️ Weather on {travel_date}: {weather_desc}, {precip}mm rain expected.")
#                     recommendations.append("🏠 Indoor alternatives recommended:")
#                     recommendations.append("   • Museums and art galleries")
#                     recommendations.append("   • Shopping malls and markets")
#                     recommendations.append("   • Restaurants and cafes")
#                     recommendations.append("   • Spa and wellness centers")
                    
#                     # Find better weather dates within ±5 days
#                     better_dates = []
#                     for i, date_str in enumerate(dates):
#                         date = datetime.strptime(date_str, "%Y-%m-%d").date()
#                         days_diff = abs((date - target_date).days)
#                         if days_diff <= 5 and days_diff > 0:
#                             wc = weather_codes[i]
#                             p = precipitation[i]
#                             ph = precip_hours[i]
#                             if wc not in bad_weather_codes and p <= 2 and ph <= 1:
#                                 better_dates.append((date_str, weather_desc_map.get(wc, f"Weather code {wc}")))
                    
#                     if better_dates:
#                         recommendations.append("📅 Better weather dates nearby:")
#                         for date_str, desc in better_dates[:3]:  # Show top 3 alternatives
#                             recommendations.append(f"   • {date_str}: {desc}")
#                 else:
#                     recommendations.append(f"✅ Great weather on {travel_date}: {weather_desc}")
#                     recommendations.append("🌟 Perfect for outdoor activities!")
#             else:
#                 recommendations.append(f"❌ Travel date {travel_date} not found in forecast range.")
#         else:
#             # General forecast analysis
#             recommendations.append(f"📍 10-day weather forecast for {city.title()}:")
#             for i in range(min(5, len(dates))):  # Show first 5 days
#                 date_str = dates[i]
#                 weather_code = weather_codes[i]
#                 weather_desc = weather_desc_map.get(weather_code, f"Weather code {weather_code}")
#                 max_temp = max_temps[i]
#                 min_temp = min_temps[i]
#                 precip = precipitation[i]
                
#                 recommendations.append(f"   • {date_str}: {weather_desc}, {min_temp}°C to {max_temp}°C, {precip}mm rain")
        
#         return "\n".join(recommendations)
        
#     except Exception as e:
#         return f"Error fetching weather data: {e}"

# # Tool 3: Enhanced Currency Conversion Tool
# @tool("Currency Conversion Tool")
# def enhanced_currency_conversion_tool(amount: float, from_currency: str, to_currency: str) -> float:
#     """
#     Converts an amount from one currency to another using a free API.
#     Returns the converted amount as a float.
#     """
#     try:
#         url = f"https://open.er-api.com/v6/latest/{from_currency}"
#         response = requests.get(url)
#         response.raise_for_status()
#         data = response.json()
#         rate = data['rates'][to_currency]
#         converted_amount = amount * rate
#         return round(converted_amount, 2)
#     except Exception as e:
#         print(f"Error converting currency: {e}")
#         return amount  # Return original amount if conversion fails

# # Tool 4: Smart Price Converter Tool
# @tool("Smart Price Converter Tool")
# def smart_price_converter_tool(price_str: str, location: str, preferred_currency: str = None) -> str:
#     """
#     Converts prices to appropriate currency based on location or user preference.
#     Input: price_str (e.g., "$50", "50 USD", "50"), location (e.g., "Mirissa, Sri Lanka")
#     Output: price in local currency or preferred currency
#     """
#     try:
#         # Extract numeric value from price string
#         import re
#         price_match = re.search(r'(\d+(?:\.\d+)?)', price_str)
#         if not price_match:
#             return price_str
        
#         amount = float(price_match.group(1))
        
#         # Determine source currency
#         if 'USD' in price_str.upper() or '$' in price_str:
#             from_currency = 'USD'
#         elif 'EUR' in price_str.upper() or '€' in price_str:
#             from_currency = 'EUR'
#         elif 'GBP' in price_str.upper() or '£' in price_str:
#             from_currency = 'GBP'
#         else:
#             from_currency = 'USD'  # Default assumption
        
#         # Determine target currency
#         if preferred_currency:
#             to_currency = preferred_currency.upper()
#         else:
#             # Get currency from location
#             country = get_country_from_city(location)
#             if country:
#                 to_currency = get_currency_from_country(country)
#             else:
#                 to_currency = 'USD'  # Default
        
#         # Convert if currencies are different
#         if from_currency != to_currency:
#             converted_amount = enhanced_currency_conversion_tool(amount, from_currency, to_currency)
            
#             # Currency symbols
#             currency_symbols = {
#                 'USD': '$', 'EUR': '€', 'GBP': '£', 'LKR': 'Rs', 'JPY': '¥',
#                 'AUD': 'A$', 'CAD': 'C$', 'CHF': 'CHF', 'CNY': '¥', 'INR': '₹'
#             }
            
#             symbol = currency_symbols.get(to_currency, to_currency)
#             return f"{symbol}{converted_amount}"
#         else:
#             return price_str
            
#     except Exception as e:
#         print(f"Error in smart price conversion: {e}")
#         return price_str

# print("Tools created successfully.")

# # Initialize LLM
# llm_model = initialize_llm()

# # Agent 1: Budget Inquiry Agent
# budget_agent = Agent(
#     role='Budget Inquiry Specialist',
#     goal='Politely ask the user for their budget if they have not provided one. The budget should be in USD.',
#     backstory='An experienced and friendly customer service representative who excels at gathering necessary client information with a warm demeanor.',
#     tools=[human_input_tool],
#     llm=llm_model,
#     allow_delegation=False,
#     verbose=True
# )

# # Agent 2: Enhanced Web Search Agent (City Expert)
# city_expert_agent = Agent(
#     role='Expert City and Activity Researcher',
#     goal='Use web search to find exciting activities, landmarks, and restaurants based on user interests in a specified city. Convert prices to local currency automatically.',
#     backstory='A seasoned travel blogger who has explored the world and knows how to find the hidden gems and must-see spots in any city. Always presents prices in the local currency for better user understanding.',
#     tools=[search_tool, smart_price_converter_tool],
#     llm=llm_model,
#     allow_delegation=False,
#     verbose=True
# )

# # Agent 3: Enhanced Local Data Agent
# local_data_agent = Agent(
#     role="Local Data Specialist",
#     goal="Provide weather-based recommendations and currency conversion insights for travel planning.",
#     backstory="Data-driven analyst using open data APIs to provide intelligent travel recommendations based on weather conditions and local currency.",
#     tools=[enhanced_weather_tool, enhanced_currency_conversion_tool],
#     llm=llm_model,
#     allow_delegation=False,
#     verbose=True
# )

# # Agent 4: Budget Verifier Agent
# budget_verifier_agent = Agent(
#     role='Budget Verification Analyst',
#     goal='Critically analyze the researched activities and their estimated costs against the user-provided budget. Provide a clear "go" or "no-go" verdict with justification.',
#     backstory='A meticulous financial analyst with a knack for sniffing out hidden costs and ensuring travel plans are financially sound. You are firm but fair.',
#     tools=[smart_price_converter_tool],
#     llm=llm_model,
#     allow_delegation=False,
#     verbose=True
# )

# # Agent 5: Enhanced General Agent (Travel Concierge)
# travel_concierge_agent = Agent(
#     role='Head Travel Concierge',
#     goal='Synthesize all the gathered information (activities, weather recommendations, budget verification) into a single, cohesive, and beautifully formatted travel itinerary.',
#     backstory='A world-class concierge from a five-star hotel, known for creating personalized and delightful travel experiences. Your final output is the masterpiece of the entire operation.',
#     tools=[],
#     llm=llm_model,
#     allow_delegation=False,
#     verbose=True
# )

# print("Agents defined successfully.")

# # --- Define User Inputs ---
# # You can change these to plan a different trip!
# location = 'Mirissa, Sri lanka'
# interests = 'entertainment, beach and villa'
# travel_date = '2025-08-15'  # Example travel date
# # Leave budget empty to see the budget_agent in action
# # budget = '1000 USD'
# budget = '25 USD'

# # Task 1: Ask for budget if not provided
# task_get_budget = Task(
#     description=f"Politely ask the user for their total budget in USD for the trip to {location} using the Human Input Tool. Return the user's response exactly as they provide it (e.g., '25 USD'). The user's interests are {interests}.",
#     expected_output="The user's total budget in USD as a string, exactly as provided by the user (e.g., '250 USD').",
#     agent=budget_agent,
#     skip_if=lambda: budget != ''
# )

# # Task 2: Find city information with automatic currency conversion
# task_find_city_info = Task(
#     description=f"Find 3-4 top attractions, activities, and restaurants in {location} that align with the interests: {interests}. Include brief descriptions and convert all prices to the local currency (LKR for Sri Lanka) using the Smart Price Converter Tool. Make sure to include the original price and converted price.",
#     expected_output="A list of attractions and activities with descriptions and prices converted to local currency. Clearly state the city name in the output.",
#     agent=city_expert_agent
# )

# # Task 3: Get weather-based recommendations
# task_get_weather_recommendations = Task(
#     description=f"Using the Enhanced Weather Tool, get weather forecast and recommendations for {location} for the travel date {travel_date}. Provide indoor alternatives and date suggestions if weather is bad.",
#     expected_output="Weather-based recommendations including indoor alternatives and date suggestions if needed.",
#     agent=local_data_agent,
#     context=[task_find_city_info]
# )

# # Task 4: Verify the budget with converted prices
# task_verify_budget = Task(
#     description=f"Analyze the findings from the city research and the user's budget, which is {budget} if provided, otherwise it's the output from the 'ask for budget' task. Sum up the estimated costs (in local currency) and compare them to the budget. Provide a clear statement on whether the plan is within budget, over budget, or if costs are unclear. Reference the specific costs found.",
#     expected_output="A concise verdict on budget feasibility, including a summary of total estimated costs vs. the available budget.",
#     agent=budget_verifier_agent,
#     context=[task_get_budget, task_find_city_info]
# )

# # Task 5: Compile the final report with weather recommendations
# task_compile_report = Task(
#     description="Create a final, human-readable travel itinerary. Combine the list of activities, weather-based recommendations, and the budget verification verdict into a single, well-formatted report. Make it sound exciting and professional. Do NOT include raw weather data or currency conversion rates in the final output - only include the weather-based recommendations and activities with converted prices.",
#     expected_output="A complete, beautiful markdown report that includes the travel plan, weather-based recommendations, and budget analysis.",
#     agent=travel_concierge_agent,
#     context=[task_verify_budget, task_get_weather_recommendations, task_find_city_info]
# )

# print("Tasks created successfully.")

# # Create the Crew
# travel_crew = Crew(
#     agents=[budget_agent, city_expert_agent, local_data_agent, budget_verifier_agent, travel_concierge_agent],
#     tasks=[task_get_budget, task_find_city_info, task_get_weather_recommendations, task_verify_budget, task_compile_report],
#     process=Process.sequential,
#     verbose=True
# )

# # Kick off the crew's work!
# result = travel_crew.kickoff(inputs={
#     'location': location, 
#     'interests': interests, 
#     'budget': budget,
#     'travel_date': travel_date
# })

# # Print the final, beautiful result
# from IPython.display import Markdown
# if hasattr(result, 'raw') and isinstance(result.raw, str):
#     display(Markdown(result.raw))
# else:
#     print(f"Error: Expected a CrewOutput object with a 'raw' attribute containing a string, but got {type(result)}")